Práctica (BigData ETls, Processing)

Asignatura: Data Science and Big Data
Máster en Intelicencia Artificial
Prof. Ronal Muresano

Notas generales: La práctica está compuesta por un conjunto de datos a los cuales se les debe hacer un conjunto de operaciones en Spark. Este ejercicio se debe documentar, explicando paso a paso cada una de las operaciones realizadas


Enunciado: 

Se tiene el siguiente dataset que mide el uso del servicio médico en Estados Unidos.

Las transformaciones que debe realizar en spark Sql son las siguientes


1) Leer el fichero desde Spark utilizando un schema predefinido.

In [0]:
Importo todas las librerias que voy a usar

In [0]:
import pyspark.sql.functions as f 
from pyspark.sql.functions import col, sum
from pyspark.sql.types import IntegerType, DoubleType, StringType, BooleanType, ArrayType, NumericType, StructType, StructField


In [0]:
Creo el esquema de data frame y cargo el dataset del fichero

In [0]:
file_path = "/FileStore/dataset2.csv"

schema = StructType([
    StructField('visits',IntegerType(), True ),
    StructField('nvisits',IntegerType(), True ),
    StructField('ovisits',IntegerType(), True ),
    StructField('novisits',IntegerType(), True ),
    StructField('emergency',IntegerType(), True ),
    StructField('hospital',IntegerType(), True ),
    StructField('health',StringType(), True ),
    StructField('chronic',IntegerType(), True ),
    StructField('adl',StringType(), True ),
    StructField('region',StringType(), True ),
    StructField('age',DoubleType(), True ),
    StructField('afam',StringType(), True ),
    StructField('gender',StringType(), True ),
    StructField('married',StringType(), True ),
    StructField('school',IntegerType(), True ),
    StructField('income',DoubleType(), True ),
    StructField('employed',StringType(), True ),
    StructField('insurance',StringType(), True ),
    StructField('medicaid',StringType(), True ),
])
dataFrame = spark.read.csv(file_path, header=True, schema = schema)

In [0]:
dataFrame.show(10)

+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+----+----+------+-------+------+------+--------+---------+--------+
|visits|nvisits|ovisits|novisits|emergency|hospital| health|chronic|    adl| region| age|afam|gender|married|school|income|employed|insurance|medicaid|
+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+----+----+------+-------+------+------+--------+---------+--------+
|     5|      0|      0|       0|        0|       1|average|      2| normal|  other| 6.9| yes|  male|    yes|     6| 2.881|     yes|      yes|      no|
|     1|      0|      2|       0|        2|       0|average|   null| normal|  other| 7.4|  no|female|    yes|    10|2.7478|      no|      yes|      no|
|    13|      0|      0|       0|        3|       3|   poor|      4|   null|  other|null| yes|female|     no|    10|0.6532|      no|       no|     yes|
|    16|      0|      5|       0|        1|       1|   poor|      2|limited|  other| 7.6

In [0]:
dataFrame.printSchema()

root
 |-- visits: integer (nullable = true)
 |-- nvisits: integer (nullable = true)
 |-- ovisits: integer (nullable = true)
 |-- novisits: integer (nullable = true)
 |-- emergency: integer (nullable = true)
 |-- hospital: integer (nullable = true)
 |-- health: string (nullable = true)
 |-- chronic: integer (nullable = true)
 |-- adl: string (nullable = true)
 |-- region: string (nullable = true)
 |-- age: double (nullable = true)
 |-- afam: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- married: string (nullable = true)
 |-- school: integer (nullable = true)
 |-- income: double (nullable = true)
 |-- employed: string (nullable = true)
 |-- insurance: string (nullable = true)
 |-- medicaid: string (nullable = true)



In [0]:
Cuanto los nulls que hay para despues verificar
Como no habia null en el dataset lo modifique y agregue algunos a mano ya que o sino no voy  apoder saber si mi relleno de nulls funciona

In [0]:

null_counts = dataFrame.select([sum(col(c).isNull().cast("int")).alias(c) for c in dataFrame.columns])
null_counts.show()


+------+-------+-------+--------+---------+--------+------+-------+---+------+---+----+------+-------+------+------+--------+---------+--------+
|visits|nvisits|ovisits|novisits|emergency|hospital|health|chronic|adl|region|age|afam|gender|married|school|income|employed|insurance|medicaid|
+------+-------+-------+--------+---------+--------+------+-------+---+------+---+----+------+-------+------+------+--------+---------+--------+
|     1|      0|      0|       0|        0|       0|     0|      2|  1|     1|  2|   0|     0|      0|     0|     0|       0|        0|       0|
+------+-------+-------+--------+---------+--------+------+-------+---+------+---+----+------+-------+------+------+--------+---------+--------+



2)	Rellenar los valores faltantes con 0 en las columnas numérica y “others” en las columnas categóricas

In [0]:
dataFrame_cols = dataFrame.dtypes
dataFrame_cols

<class 'list'>
Out[36]: [('visits', 'int'),
 ('nvisits', 'int'),
 ('ovisits', 'int'),
 ('novisits', 'int'),
 ('emergency', 'int'),
 ('hospital', 'int'),
 ('health', 'string'),
 ('chronic', 'int'),
 ('adl', 'string'),
 ('region', 'string'),
 ('age', 'double'),
 ('afam', 'string'),
 ('gender', 'string'),
 ('married', 'string'),
 ('school', 'int'),
 ('income', 'double'),
 ('employed', 'string'),
 ('insurance', 'string'),
 ('medicaid', 'string')]

In [0]:
Creo una lista con todos los tipos de las columnas para despues poder identificar si es numerica o no numerica en el if
Recorro todas las columnas y voy comparando con los tipos de columna, asi identifico si es numerica, si es numerica guar un 0 y si no "others"

In [0]:
values = {}

for col, col_type in dataFrame_cols:
    if col_type in ['int', 'double']:
        values[col] = 0
    else:
        values[col] = 'others'

values


Out[26]: {'visits': 0,
 'nvisits': 0,
 'ovisits': 0,
 'novisits': 0,
 'emergency': 0,
 'hospital': 0,
 'health': 'others',
 'chronic': 0,
 'adl': 'others',
 'region': 'others',
 'age': 0,
 'afam': 'others',
 'gender': 'others',
 'married': 'others',
 'school': 0,
 'income': 0,
 'employed': 'others',
 'insurance': 'others',
 'medicaid': 'others'}

In [0]:
uso fillna con el dictionario indicandole en que columna debe usar 0 y en cual "others"

In [0]:
dataFrame_filled = dataFrame.fillna(values)
dataFrame_filled.show(20)

+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+
|visits|nvisits|ovisits|novisits|emergency|hospital| health|chronic|    adl| region|age|afam|gender|married|school|  income|employed|insurance|medicaid|
+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+
|     5|      0|      0|       0|        0|       1|average|      2| normal|  other|6.9| yes|  male|    yes|     6|   2.881|     yes|      yes|      no|
|     1|      0|      2|       0|        2|       0|average|      0| normal|  other|7.4|  no|female|    yes|    10|  2.7478|      no|      yes|      no|
|    13|      0|      0|       0|        3|       3|   poor|      4| others|  other|0.0| yes|female|     no|    10|  0.6532|      no|       no|     yes|
|    16|      0|      5|       0|        1|       1|   poor|      2|limited|  othe

In [0]:
Cuento los nulls en el dataframe filled para comprobar que no hay nulls

In [0]:
null_counts = dataFrame_filled.select([sum(col(c).isNull().cast("int")).alias(c) for c in dataFrame_filled.columns])
null_counts.show()

+------+-------+-------+--------+---------+--------+------+-------+---+------+---+----+------+-------+------+------+--------+---------+--------+
|visits|nvisits|ovisits|novisits|emergency|hospital|health|chronic|adl|region|age|afam|gender|married|school|income|employed|insurance|medicaid|
+------+-------+-------+--------+---------+--------+------+-------+---+------+---+----+------+-------+------+------+--------+---------+--------+
|     0|      0|      0|       0|        0|       0|     0|      0|  0|     0|  0|   0|     0|      0|     0|     0|       0|        0|       0|
+------+-------+-------+--------+---------+--------+------+-------+---+------+---+----+------+-------+------+------+--------+---------+--------+



3)	Calcular el número de visitas totales y asignarlas a una nueva columna llamada TotalVisits

In [0]:
dataFrame_total_visits = dataFrame_filled.withColumn("TotalVisits", col("visits")+col("nvisits")+col("ovisits")+col("emergency"))
dataFrame_total_visits.show(20)

+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+
|visits|nvisits|ovisits|novisits|emergency|hospital| health|chronic|    adl| region|age|afam|gender|married|school|  income|employed|insurance|medicaid|TotalVisits|
+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+
|     5|      0|      0|       0|        0|       1|average|      2| normal|  other|6.9| yes|  male|    yes|     6|   2.881|     yes|      yes|      no|          5|
|     1|      0|      2|       0|        2|       0|average|      0| normal|  other|7.4|  no|female|    yes|    10|  2.7478|      no|      yes|      no|          5|
|    13|      0|      0|       0|        3|       3|   poor|      4| others|  other|0.0| yes|female|     no|    10|  0.6532|      no|       no|     yes|         16|
|    16|  

In [0]:
4)	Cambiar la variable gender por F/M dependiendo del caso

In [0]:
dataFrame_total_visits = dataFrame_total_visits.withColumn('gender', 
    f.when(f.col("gender") == 'female', 'F').otherwise('M'))

dataFrame_total_visits.show(20)


+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+
|visits|nvisits|ovisits|novisits|emergency|hospital| health|chronic|    adl| region|age|afam|gender|married|school|  income|employed|insurance|medicaid|TotalVisits|
+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+
|     5|      0|      0|       0|        0|       1|average|      2| normal|  other|6.9| yes|     M|    yes|     6|   2.881|     yes|      yes|      no|          5|
|     1|      0|      2|       0|        2|       0|average|      0| normal|  other|7.4|  no|     F|    yes|    10|  2.7478|      no|      yes|      no|          5|
|    13|      0|      0|       0|        3|       3|   poor|      4| others|  other|0.0| yes|     F|     no|    10|  0.6532|      no|       no|     yes|         16|
|    16|  

5)	Crear una variable categórica llamada School_training que se formará con las siguientes condiciones: 

Si el valor está entre 0 y 4	Low_trained
Si el valor está es mayor a 4 y menor que 8 	Mid_trained
Si el valor es mayor que 8 entonces es	High trained


In [0]:
dataFrame_total_visits = dataFrame_total_visits.withColumn('School_training', 
    f.when(f.col('school') <= 4, 'Low_trained').when(
    (f.col('school') > 4) & (f.col('school') < 8), 'Mid_trained').otherwise( 'High_trained'))

dataFrame_total_visits.show(20)




+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+---------------+
|visits|nvisits|ovisits|novisits|emergency|hospital| health|chronic|    adl| region|age|afam|gender|married|school|  income|employed|insurance|medicaid|TotalVisits|School_training|
+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+---------------+
|     5|      0|      0|       0|        0|       1|average|      2| normal|  other|6.9| yes|     M|    yes|     6|   2.881|     yes|      yes|      no|          5|    Mid_trained|
|     1|      0|      2|       0|        2|       0|average|      0| normal|  other|7.4|  no|     F|    yes|    10|  2.7478|      no|      yes|      no|          5|   High_trained|
|    13|      0|      0|       0|        3|       3|   poor|      4| others|  other|0.0| yes|  

6)	Crear una clave compuesta concatenando las siguientes columnas age, health y region

In [0]:
dataFrame_total_visits_key = dataFrame_total_visits.withColumn('key', f.concat(col('age'), col('health'), col('region')))

dataFrame_total_visits_key.show(20)

+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+---------------+-----------------+
|visits|nvisits|ovisits|novisits|emergency|hospital| health|chronic|    adl| region|age|afam|gender|married|school|  income|employed|insurance|medicaid|TotalVisits|School_training|              key|
+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+---------------+-----------------+
|     5|      0|      0|       0|        0|       1|average|      2| normal|  other|6.9| yes|     M|    yes|     6|   2.881|     yes|      yes|      no|          5|    Mid_trained|  6.9averageother|
|     1|      0|      2|       0|        2|       0|average|      0| normal|  other|7.4|  no|     F|    yes|    10|  2.7478|      no|      yes|      no|          5|   High_trained|  7.4averageother|
|    

7)	Crear un identificar único por cada registro (Ayuda, monolitycal_id)

In [0]:
dataFrame_total_visits_key_mid = dataFrame_total_visits_key.withColumn('uid', f.monotonically_increasing_id())
dataFrame_total_visits_key_mid.show(20)

+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+---------------+-----------------+---+
|visits|nvisits|ovisits|novisits|emergency|hospital| health|chronic|    adl| region|age|afam|gender|married|school|  income|employed|insurance|medicaid|TotalVisits|School_training|              key|uid|
+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+---------------+-----------------+---+
|     5|      0|      0|       0|        0|       1|average|      2| normal|  other|6.9| yes|     M|    yes|     6|   2.881|     yes|      yes|      no|          5|    Mid_trained|  6.9averageother|  0|
|     1|      0|      2|       0|        2|       0|average|      0| normal|  other|7.4|  no|     F|    yes|    10|  2.7478|      no|      yes|      no|          5|   High_trained|  7.4ave

8)	Cambiar las columnas employed, insurance and Medicaid por valores T/F

In [0]:
dataFrame_total_visits_key_mid = dataFrame_total_visits_key_mid.withColumn('employed', f.when((f.col("employed") == 'yes'), 'T').otherwise('F'))
dataFrame_total_visits_key_mid = dataFrame_total_visits_key_mid.withColumn('insurance', f.when((f.col("insurance") == 'yes'), 'T').otherwise('F'))   
dataFrame_total_visits_key_mid = dataFrame_total_visits_key_mid.withColumn('medicaid', f.when((f.col("medicaid") == 'yes'), 'T').otherwise('F'))       

dataFrame_total_visits_key_mid.show(20)

+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+---------------+-----------------+---+
|visits|nvisits|ovisits|novisits|emergency|hospital| health|chronic|    adl| region|age|afam|gender|married|school|  income|employed|insurance|medicaid|TotalVisits|School_training|              key|uid|
+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+---------------+-----------------+---+
|     5|      0|      0|       0|        0|       1|average|      2| normal|  other|6.9| yes|     M|    yes|     6|   2.881|       T|        T|       F|          5|    Mid_trained|  6.9averageother|  0|
|     1|      0|      2|       0|        2|       0|average|      0| normal|  other|7.4|  no|     F|    yes|    10|  2.7478|       F|        T|       F|          5|   High_trained|  7.4ave

9)	Crear un fichero parquet de los datos calculados en la columna school_training 

In [0]:

dataFrame_total_visits_key_mid.select('school_training').write.mode('overwrite').parquet('school_training')

10)	Leer el fichero y hacer un show para comprobar los datos

In [0]:
df_school_training = spark.read.parquet('dbfs:/school_training/part-00000-tid-5091758245159236913-484a3df2-2fb9-4a02-a365-c65f90df4dee-42-1-c000.snappy.parquet')

df_school_training.show(20)

+---------------+
|school_training|
+---------------+
|    Mid_trained|
|   High_trained|
|   High_trained|
|    Low_trained|
|    Mid_trained|
|    Mid_trained|
|   High_trained|
|   High_trained|
|   High_trained|
|   High_trained|
|   High_trained|
|   High_trained|
|   High_trained|
|   High_trained|
|   High_trained|
|   High_trained|
|   High_trained|
|   High_trained|
|   High_trained|
|   High_trained|
+---------------+
only showing top 20 rows



11)	Agrupar las columnas edad y región y hacer un count, average y una suma de la columna Chronic

In [0]:
dataFrame_grouped = dataFrame_total_visits_key_mid.groupBy('age', 'region').agg(f.count('Chronic').alias('Count'),
                                                                   f.avg('Chronic').alias('Avg'),
                                                                   f.sum('Chronic').alias('Sum'))

dataFrame_grouped.show(20)

+---+---------+-----+------------------+---+
|age|   region|Count|               Avg|Sum|
+---+---------+-----+------------------+---+
|6.8|northeast|   63|1.4285714285714286| 90|
|8.2|northeast|   24|             1.625| 39|
|9.5|  midwest|    1|               1.0|  1|
|9.0|  midwest|    7|1.8571428571428572| 13|
|7.2|     west|   45|1.0888888888888888| 49|
|7.0|northeast|   60|1.6166666666666667| 97|
|7.9|     west|   24|             1.375| 33|
|8.4|  midwest|   18|1.0555555555555556| 19|
|9.1|     west|    6|1.6666666666666667| 10|
|8.5|     west|   15|               1.4| 21|
|8.3|    other|   44|2.0681818181818183| 91|
|8.0|northeast|    9|0.7777777777777778|  7|
|9.2|northeast|    3|1.6666666666666667|  5|
|6.8|     west|   55|1.6363636363636365| 90|
|8.3|     west|   17| 2.176470588235294| 37|
|8.9|    other|   11|2.4545454545454546| 27|
|7.2|northeast|   55| 1.490909090909091| 82|
|6.9|northeast|   58| 1.293103448275862| 75|
|9.1|northeast|    1|               0.0|  0|
|9.6|     

12)	Cree un nuevo dataset con un par de columnas y haga el join inner con el mismo

In [0]:
dataFrame_new = dataFrame_total_visits_key_mid.select('married', 'employed', 'uid').withColumnRenamed('married', 'have_children').withColumnRenamed('employed', 'have_car')

dataFrame_new.show(20)

+-------------+--------+---+
|have_children|have_car|uid|
+-------------+--------+---+
|          yes|       T|  0|
|          yes|       F|  1|
|           no|       F|  2|
|          yes|       F|  3|
|          yes|       F|  4|
|           no|       F|  5|
|           no|       F|  6|
|           no|       F|  7|
|           no|       F|  8|
|           no|       F|  9|
|          yes|       T| 10|
|          yes|       F| 11|
|           no|       F| 12|
|           no|       F| 13|
|           no|       F| 14|
|           no|       F| 15|
|          yes|       F| 16|
|          yes|       F| 17|
|          yes|       F| 18|
|           no|       F| 19|
+-------------+--------+---+
only showing top 20 rows



In [0]:
dataFrame_joined = dataFrame_total_visits_key_mid.join(dataFrame_new, "uid", "inner")

dataFrame_joined.show(20)

+---+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+---------------+-----------------+-------------+--------+
|uid|visits|nvisits|ovisits|novisits|emergency|hospital| health|chronic|    adl| region|age|afam|gender|married|school|  income|employed|insurance|medicaid|TotalVisits|School_training|              key|have_children|have_car|
+---+------+-------+-------+--------+---------+--------+-------+-------+-------+-------+---+----+------+-------+------+--------+--------+---------+--------+-----------+---------------+-----------------+-------------+--------+
|  0|     5|      0|      0|       0|        0|       1|average|      2| normal|  other|6.9| yes|     M|    yes|     6|   2.881|       T|        T|       F|          5|    Mid_trained|  6.9averageother|          yes|       T|
|  1|     1|      0|      2|       0|        2|       0|average|      0| normal|  other|7.4|  no

13)	Generar una función UDF que permita calcular la concatenación de dos columnas

In [0]:
def cal_concat(df, col_name1, col_name2):
    df_concat = df.withColumn('concat', f.concat(col(col_name1), col(col_name2)))
    return df_concat.select('concat')

cal_concat(dataFrame_joined, 'age', 'have_car').show(20)

+------+
|concat|
+------+
|  6.9T|
|  7.4F|
|  0.0F|
|  7.6F|
|  7.9F|
|  6.6F|
|  0.0F|
|  8.7F|
|  7.3F|
|  7.8F|
|  6.6T|
|  6.9F|
|  8.4F|
|  8.1F|
|  7.8F|
|  7.6F|
|  8.3F|
|  8.0F|
|  7.3F|
|  7.4F|
+------+
only showing top 20 rows



14)	Genere el fichero resultante en parquet y en formato snappy.



In [0]:
cal_concat(dataFrame_joined, 'age', 'have_car').write.mode('overwrite').option("compression", "snappy").parquet('concat')